In [3]:
import talib
import pandas as pd

In [51]:
ticker_data = pd.read_csv('/Users/liuda/Local/data/finance/stocks/ticker/history/AMZN.csv')

In [52]:
ticker_data['Adj High'] = ticker_data['High'] + ticker_data['Adj Close'] - ticker_data['Close']
ticker_data['Adj Low'] = ticker_data['Low'] + ticker_data['Adj Close'] - ticker_data['Close']
ticker_data['Adj Open'] = ticker_data['Open'] + ticker_data['Adj Close'] - ticker_data['Close']


In [53]:
ticker_data = ticker_data.sort_values(by="Date").reset_index(drop=True)
ticker_data['qt'] = ticker_data['Date'].apply(pd.to_datetime).dt.to_period('Q').astype(str).str.replace('Q', 'q')


In [55]:
ticker_data = ticker_data.sort_values(by='Date').reset_index(drop=True)


In [56]:
ticker_data['DateIndex'] = pd.to_datetime(ticker_data['Date'])


In [58]:
ticker_data['atr'] = talib.ATR(ticker_data['Adj High'], ticker_data['Adj Low'], ticker_data['Adj Close'], timeperiod=14)


In [60]:
indicators = [
    (talib.EMA, "EMA_13_day", 13),
    (talib.EMA, "EMA_21_day", 20),
    (talib.EMA, "EMA_144_day", 144),
    (talib.EMA, "EMA_169_day", 169),
    (talib.SMA, "SMA_5_day", 5),
    (talib.SMA, "SMA_10_day", 10),
    (talib.SMA, "SMA_20_day", 20),
    (talib.SMA, "SMA_50_day", 50),
    (talib.SMA, "SMA_100_day", 100),
]
                                                                          
                                                                          
for func, name, period in indicators:
    ticker_data[name] = func(ticker_data['Adj Close'], timeperiod=period)


In [62]:
resample_rules = [("W", "weekly"), ("ME", "monthly")]
                                                      
ticker_data.set_index('DateIndex', inplace=True)


In [67]:
ticker_data_fill_30_day = ticker_data.copy(deep=True)
future_dates = pd.date_range(start=ticker_data_fill_30_day.index[-1] + pd.Timedelta(days=1), periods=30)
ticker_data_fill_30_day = ticker_data_fill_30_day.reindex(ticker_data_fill_30_day.index.union(future_dates))
ticker_data_fill_30_day = ticker_data_fill_30_day.ffill()

In [72]:
for rule, label in resample_rules:
    resampled_data = ticker_data_fill_30_day.resample(   # 从fill 30 day 采样，避免尾部值缺失
        rule, label="left", offset="1D"
    ).agg(
        {
            "Date": "first",
            "Open": "first",
            "High": "max",
            "Low": "min",
            "Adj High": "max",
            "Adj Open": "first",
            "Adj Low": "min",
            "Adj Close": "last",
            "Volume": "sum",
        }
    )
                                                                                                               
    resampled_indicators = [
        (talib.SMA, f"SMA_20_{label}", 20),
        (talib.SMA, f"SMA_50_{label}", 50),
    ]
    for func, name, period in resampled_indicators:
        resampled_data[name] = func(resampled_data["Adj Close"], timeperiod=period)
        # resampled_data = strategy_utils.cal_direction(resampled_data, name, f"Direct_{name}")
                                                                                                               
    # 方便采样，按day补齐
    resampled_data = resampled_data.resample("D").ffill()
                                                                                                               
    cols = [item[1] for item in resampled_indicators] + [f"Direct_{item[1]}" for item in resampled_indicators]
    cols.append('Date')
    ticker_data = ticker_data.merge(resampled_data[cols], on='Date', how='outer', suffixes=('', '_y'))

KeyError: "['Direct_SMA_20_weekly', 'Direct_SMA_50_weekly'] not in index"

In [69]:
ticker_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3770 entries, 2010-01-19 to 2025-01-10
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         3770 non-null   object 
 1   Open         3770 non-null   float64
 2   High         3770 non-null   float64
 3   Low          3770 non-null   float64
 4   Close        3770 non-null   float64
 5   Adj Close    3770 non-null   float64
 6   Volume       3770 non-null   int64  
 7   Adj High     3770 non-null   float64
 8   Adj Low      3770 non-null   float64
 9   Adj Open     3770 non-null   float64
 10  qt           3770 non-null   object 
 11  atr          3756 non-null   float64
 12  EMA_13_day   3758 non-null   float64
 13  EMA_21_day   3751 non-null   float64
 14  EMA_144_day  3627 non-null   float64
 15  EMA_169_day  3602 non-null   float64
 16  SMA_5_day    3766 non-null   float64
 17  SMA_10_day   3761 non-null   float64
 18  SMA_20_day   3751 non-null   f